### Importing Necessary Libraries

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton, QMessageBox, QFileDialog, QVBoxLayout, QHBoxLayout
from PyQt5.QtGui import QPixmap
from PyQt5 import QtCore
import tensorflow as tf
import numpy as np
import cv2
import sys

### Creating buttons for image uploading (in jpg, jpeg and png formats), prediction and quiting and labels to visualize image and its predicted disease (healthy if tomato has no disease)

In [ ]:
class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setGeometry(200, 200, 600, 600)
        self.setWindowTitle('Tomato Disease Classification')
        self.initUI()
    
    def initUI(self):
        self.image_label = QLabel()
        self.predicted_label = QLabel()

        self.upload_button = QPushButton('Upload Image')
        self.upload_button.clicked.connect(self.click_upload)
        self.upload_button.setFixedSize(QtCore.QSize(100, 25))
        
        self.predict_button = QPushButton('Predict')
        self.predict_button.clicked.connect(self.click_predict)
        self.predict_button.setFixedSize(QtCore.QSize(100, 25))

        self.quit_button = QPushButton('Quit')
        self.quit_button.clicked.connect(self.click_quit)
        self.quit_button.setFixedSize(QtCore.QSize(100, 25))

        self.vbox = QVBoxLayout()
        self.vbox.addWidget(self.upload_button, 0, alignment=QtCore.Qt.AlignHCenter)
        self.vbox.addWidget(self.predicted_label, 0, alignment=QtCore.Qt.AlignHCenter)
        self.vbox.addWidget(self.image_label, 0, alignment=QtCore.Qt.AlignHCenter)
        self.vbox.addWidget(self.predict_button, 0, alignment=QtCore.Qt.AlignHCenter)
        self.vbox.addWidget(self.quit_button, 0, alignment=QtCore.Qt.AlignHCenter)

        self.hbox = QHBoxLayout()
        self.hbox.addStretch()
        self.hbox.addLayout(self.vbox)
        self.hbox.addStretch()

        self.setLayout(self.hbox)
        self.show()

    def get_model(self):
        self.model = tf.keras.models.load_model('model_tomato.h5')
        return self.model

    def process_image(self, img):
        img = cv2.resize(img, (64, 64))
        img = np.expand_dims(img , axis=0)
        return img

    def click_predict(self):
        model = self.get_model()
        image = self.process_image(self.image)
        pred = model.predict(image)
        label = self.decoder(pred)
        self.predicted_label.setText(f'Predicted: {label}')
        self.predicted_label.adjustSize()

    def decoder(self, label):
        pred = np.argmax(label, axis=1)
        dic = {0:'Bacterial_spot', 1:'Early_blight', 2:'Late_blight', 3:'Leaf_Mold', 4:'Septoria_leaf_spot', 5:'Spider_mites Two-spotted_spider_mite',\
            6:'Target_Spot', 7:'Tomato_Yellow_Leaf_Curl_Virus', 8:'Tomato_mosaic_virus', 9:'healthy'}
        return dic[pred[0]]

    def closeEvent(self, event):
        close_reply=QMessageBox.question(self,'Message','Are you sure you want to quit?',\
                                         QMessageBox.Yes|QMessageBox.No,QMessageBox.No )
        if close_reply==QMessageBox.Yes:
            event.accept()
        else:
            event.ignore()
    
    def click_quit(self):
        quit_reply=QMessageBox.question(self,'Message','Are you sure you want to quit?',\
                                        QMessageBox.Yes|QMessageBox.No,QMessageBox.No )
        if quit_reply==QMessageBox.Yes:
            QApplication.instance().quit()
        else:
            pass

   
    def click_upload(self):
        filename = QFileDialog.getOpenFileName(self, 'Open File', '.', "Image file(*.jpg *.jpeg *.png *bmp)")
        imagepath = filename[0]
        self.image = cv2.imread(imagepath)
        self.pixmap = QPixmap(imagepath)
        self.pixmap= self.pixmap.scaled(512, 512, QtCore.Qt.KeepAspectRatio)
        self.image_label.setPixmap(self.pixmap)

### Creating and calling our main function to actually create our GUI

In [ ]:
def main():
    app = QApplication(sys.argv)
    win = MainWindow()
    sys.exit(app.exec_())

In [ ]:
if __name__ == '__main__':
    main()